# PyTorch Classification 

This notebook is a basic example for training and testing CNN model with PyTorch.

## Defining the PyTorch model and runner

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import json
import numpy as np
import datetime
from tqdm.notebook import tqdm

In [2]:
class ConvBlock(nn.Module):
    def __init__(self, in_filters, out_filters, kernel_size=3, batchnorm=True, padding=1):
        super(ConvBlock, self).__init__()
        self.c1 = nn.Conv2d(in_filters, out_filters, kernel_size, padding=padding)
        self.bn = batchnorm
        self.b1 = nn.BatchNorm2d(out_filters)
        
    def forward(self, x):
        x = self.c1(x)
        if self.bn: 
            x = self.b1(x)
        return x
    
class Net(nn.Module):
    def __init__ (self, imgsize=32, num_classes=10, in_filters=1, filters=8, kernel_size=3, batchnorm=True, activ=F.relu, padding=1):
        super(Net, self).__init__()

        self.activ = activ
        self.bn = batchnorm
        self.imgsize = imgsize
        
        # Convolutional layers
        self.activ = activ
        self.conv1 = ConvBlock(in_filters=in_filters, out_filters=filters, 
                               kernel_size=kernel_size, batchnorm=batchnorm, padding=padding)
        self.conv2 = ConvBlock(in_filters=filters, out_filters=filters, 
                               kernel_size=kernel_size, batchnorm=batchnorm, padding=padding)
        
        self.fc1 = nn.Linear(filters * imgsize ** 2, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.activ(self.conv1(x))
        x = self.activ(self.conv2(x))
        
        x = x.view(-1, self.num_flat_features(x))
        x = self.activ(self.fc1(x))
        x = self.fc2(x)
        return x
        
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
class Trainer:
    def __init__ (self, imgsize=32, num_classes=10, in_filters=1, filters=8, kernel_size=3, batchnorm=True, activ=F.relu):
        """
        Trainer for the CNN

        @num_epochs: Number of epochs
        @lr: initial learning rate
        @batch_size: Size of single batch
        @decay: decay LR by 3 after these many epochs
        """
        self.imgsize = imgsize
        self.net = Net(imgsize=imgsize, num_classes=num_classes, in_filters=in_filters, filters=filters, kernel_size=kernel_size,
                       batchnorm=batchnorm, activ=activ)
        
        self.cuda_flag = torch.cuda.is_available()
        if self.cuda_flag:
            self.net = self.net.cuda()

    def train(self, X, Y, epochs=10, lr=0.001, batch_size=64, decay=5000, logging=True, log_file=None):
        """
        Train the CNN

        Params:
        @X: Training data - input of the model
        @Y: Training labels
        @logging: True for printing the training progress after each epoch
        @log_file: Path of log file
        """
        X = [x.reshape(1, self.imgsize, self.imgsize) for x in X]
        
        inputs = torch.FloatTensor(X)
        labels = torch.LongTensor(Y)
        
        train_dataset = TensorDataset(inputs, labels)
        trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        self.net.train()
        criterion = nn.CrossEntropyLoss()

        for epoch in range(1, epochs+1): # loop over data multiple times
            # Decreasing the learning rate
            if (epoch % decay == 0):
                lr /= 3
                
            optimizer = optim.SGD(self.net.parameters(), lr=lr, momentum=0.9)
            
            tot_loss = 0.0
            for data in tqdm(trainloader):
                # get the inputs
                inputs, labels = data
                if self.cuda_flag:
                    inputs = inputs.cuda()
                    labels = labels.cuda()

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                o = self.net(inputs)
                loss = criterion(o, labels)

                loss.backward()
                optimizer.step()
                
                tot_loss += loss.item()
                
            tot_loss /= len(trainloader)

            # logging statistics
            timestamp = str(datetime.datetime.now()).split('.')[0]
            log = json.dumps({
                'timestamp': timestamp,
                'epoch': epoch,
                'loss': float('%.7f' % tot_loss),
                'lr': float('%.6f' % lr)
            })
            if logging:
                print (log)

            if log_file is not None:
                with open(log_file, 'a') as f:
                    f.write("{}\n".format(log))
            
        print ('Finished Training')

    def predict(self, inputs):
        """
        Predict the task labels corresponding to the input images
        """
        inputs = [x.reshape(1, self.imgsize, self.imgsize) for x in inputs]
        inputs = torch.FloatTensor(inputs)
        if self.cuda_flag:
            inputs = inputs.cuda()

        self.net.eval()
        with torch.no_grad():
            labels = self.net(inputs).cpu().numpy()
            
        return np.argmax(labels, axis=1)

    def score(self, X, Y):
        """
        Score the model -- compute accuracy
        """
        pred = self.predict(X)
        acc = np.sum(pred == Y) / len(Y)
        return float(acc)

    def save_model(self, checkpoint_path, model=None):
        if model is None: model = self.net
        torch.save(model.state_dict(), checkpoint_path)
    
    def load_model(self, checkpoint_path, model=None):
        if model is None: model = self.net
        if self.cuda_flag:
            model.load_state_dict(torch.load(checkpoint_path))
        else:
            model.load_state_dict(torch.load(checkpoint_path, map_location=torch.device('cpu')))


## Get the Data

In [4]:
import pickle
import os
from subprocess import call

%pip install root_numpy
from root_numpy import root2array

Note: you may need to restart the kernel to use updated packages.
Welcome to JupyROOT 6.21/01


In [5]:
def get_dataset(data_file, test_split=0.2):
    """
    @data_file: path of the root file
    @test_split: fraction of data to be used as test set
    """
    assert 0 <= test_split <= 0.3

    # Load data
    signal = root2array(data_file, 'sig_tree')
    background = root2array(data_file, 'bkg_tree')

    tree_data = [
        np.array([img[0] for img in signal]),
        np.array([img[0] for img in background])
    ]

    X_train = []
    Y_train = []
    X_test = []
    Y_test = []

    # Deterministic Random
    np.random.seed(0)

    for label, data in enumerate(tree_data):
        np.random.shuffle(data)

        test_size = int(len(data) * test_split)
        X_train.append(data[:-test_size])
        X_test.append(data[-test_size:])
        Y_train.append([label] * (len(data) - test_size))
        Y_test.append([label] * test_size)

    X_train = np.concatenate(X_train, axis=0)
    X_test = np.concatenate(X_test, axis=0)
    Y_train = np.concatenate(Y_train)
    Y_test = np.concatenate(Y_test)

    assert len(Y_train) == len(X_train)
    assert len(Y_test) == len(X_test)

    # Shuffling the data
    train_perm = np.random.permutation(len(X_train))
    X_train = X_train[train_perm]
    Y_train = Y_train[train_perm]
    
    test_perm = np.random.permutation(len(X_test))
    X_test = X_test[test_perm]
    Y_test = Y_test[test_perm]
    
    return (X_train, Y_train), (X_test, Y_test)

In [6]:
# Load data
inputFile = 'sample_images_32x32.root'
if not os.path.isfile(inputFile):
    call(['curl', '-o', inputFile, 'https://cernbox.cern.ch/index.php/s/mba2sFJ3ugoy269/download'])

(X_train, Y_train), (X_test, Y_test) = get_dataset(inputFile, test_split=0.2)

## Train the model

In [7]:
# Model config
lr = 0.0003
num_epochs = 20
batch_size = 64

In [8]:
trainer = Trainer(imgsize=32, num_classes=2)
trainer.train(X_train, Y_train, lr=lr, epochs=num_epochs, batch_size=batch_size)
trainer.save_model('model_cnn.pth')


{"timestamp": "2020-03-17 15:06:09", "epoch": 1, "loss": 0.671591, "lr": 0.0003}



{"timestamp": "2020-03-17 15:06:13", "epoch": 2, "loss": 0.591326, "lr": 0.0003}



{"timestamp": "2020-03-17 15:06:17", "epoch": 3, "loss": 0.4519308, "lr": 0.0003}



{"timestamp": "2020-03-17 15:06:21", "epoch": 4, "loss": 0.2858451, "lr": 0.0003}



{"timestamp": "2020-03-17 15:06:25", "epoch": 5, "loss": 0.1693994, "lr": 0.0003}


...


{"timestamp": "2020-03-17 15:07:08", "epoch": 16, "loss": 0.015077, "lr": 0.0003}



{"timestamp": "2020-03-17 15:07:11", "epoch": 17, "loss": 0.0140529, "lr": 0.0003}



{"timestamp": "2020-03-17 15:07:15", "epoch": 18, "loss": 0.0125363, "lr": 0.0003}



{"timestamp": "2020-03-17 15:07:19", "epoch": 19, "loss": 0.0119496, "lr": 0.0003}



{"timestamp": "2020-03-17 15:07:23", "epoch": 20, "loss": 0.0114691, "lr": 0.0003}
Finished Training


## Test the model

In [9]:
print ('Evaluating...')
print ('Training Acc.: {:.4f} %'.format(trainer.score(X_train, Y_train) * 100))
print ('Test Acc.    : {:.4f} %'.format(trainer.score(X_test, Y_test) * 100))

Evaluating...
Training Acc.: 99.9875 %
Test Acc.    : 99.9750 %
